In [1]:
import os
import pandas as pd

DATA_SET_DIR = './project2_data/'
TRAINING_DATA_PATH = DATA_SET_DIR+'olid-training-v1_0.csv'
TEST_A_DATA_PATH = DATA_SET_DIR+'testset-levela.tsv'

train = pd.read_csv(TRAINING_DATA_PATH,sep='\t', index_col='id')
train.head(3)

,tweet,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN


In [2]:
taskA_describe = train.subtask_a.value_counts()
print(taskA_describe)

NOT    8840
OFF    4400
Name: subtask_a, dtype: int64


In [3]:
test = pd.read_csv(TEST_A_DATA_PATH,sep='\t', index_col='id')
print(test.head(3))

                                                   tweet
id                                                      
15923  #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
27014  #ConstitutionDay is revered by Conservatives, ...
30530  #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...


In [4]:
print(train.isna().any())
print(test.isna().any())

tweet        False
subtask_a    False
subtask_b     True
subtask_c     True
dtype: bool
tweet    False
dtype: bool


In [5]:
from pycontractions import Contractions
cont = Contractions(api_key="glove-twitter-25")

In [6]:
import time
start_t=time.time()
print(list(cont.expand_texts([str_list[0]],precise=True)))
end_t=time.time()
print(end_t-start_t)

NameError: name 'str_list' is not defined

In [7]:
import re, string
legal_chars = string.printable
tmp_re = re.compile('[’]')
regex = re.compile('[^%s]' % re.escape(legal_chars))
# def test_re(s):  # From Vinko's solution, with fix.
#     return regex.sub('', s)
# str1='@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊'
# test_re(str1)
tmp_re = re.compile('[’]')
train.tweet = train.tweet.apply(lambda x: tmp_re.sub("'", x))
test.tweet = test.tweet.apply(lambda x: tmp_re.sub("'", x))
train.tweet = train.tweet.apply(lambda x: regex.sub(' ', x))
test.tweet = test.tweet.apply(lambda x: regex.sub(' ', x))
print(pd.DataFrame(train.head(5)))
print(pd.DataFrame(train.head(5)))
print(train.isna().any())
print(test.isna().any())

                                                   tweet subtask_a subtask_b  \
id                                                                             
86426  @USER She should ask a few native Americans wh...       OFF       UNT   
90194  @USER @USER Go home you're drunk!!! @USER #MAG...       OFF       TIN   
16820  Amazon is investigating Chinese employees who ...       NOT       NaN   
62688  @USER Someone should'veTaken" this piece of sh...       OFF       UNT   
43605  @USER @USER Obama wanted liberals &amp; illega...       NOT       NaN   

      subtask_c  
id               
86426       NaN  
90194       IND  
16820       NaN  
62688       NaN  
43605       NaN  
                                                   tweet subtask_a subtask_b  \
id                                                                             
86426  @USER She should ask a few native Americans wh...       OFF       UNT   
90194  @USER @USER Go home you're drunk!!! @USER #MAG...       OFF       

In [56]:
import numpy as np
def expand_contra(text):
    text_list = list(cont.expand_texts([text],precise=True))
    assert (len(text_list)==1),'error!'
    return text_list[0]
def process(data):
    res = data.apply(expand_contra)
    return res

def check_merge_idx(data, res):
    assert (data.index == res.index).all(), 'Something error when merge data'

def parallelize(data, func):
    from multiprocessing import cpu_count, Pool
    cores = partitions = cpu_count()
    data_split = np.array_split(data, partitions)
    pool = Pool(cores)
    res = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    check_merge_idx(data, res)
    return res

In [57]:
print("start to training")
train['tweet_exp'] = parallelize(train.loc[:,"tweet"], process)

start to training


In [65]:
print("start to training")
test['tweet_exp'] = parallelize(test.loc[:,"tweet"], process)

start to training


In [74]:
DATA_PATH = DATA_SET_DIR+'testset-levelc.tsv'
data = pd.read_csv(DATA_PATH,sep='\t', index_col='id')
print(data.head(3))
print(data.isna().any())

                                                   tweet
id                                                      
15923  #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
60133  #NoPasaran: Unity demo to oppose the far-right...
83681           . . . What the fuck did he do this time?
tweet    False
dtype: bool


In [75]:
data.tweet = data.tweet.apply(lambda x: tmp_re.sub("'", x))
data.tweet = data.tweet.apply(lambda x: regex.sub(' ', x))
print("start to training")
data['tweet_exp'] = parallelize(data.loc[:,"tweet"], process)
cols = ['tweet_exp']
data = data.loc[:, cols]
data.to_csv(DATA_SET_DIR+'testset-levelc-clean.tsv',sep='\t',index=True)

start to training


In [68]:
TESTing_PATH = DATA_SET_DIR+'olid-training-clean.csv'

train_t = pd.read_csv(TESTing_PATH,sep='\t', index_col='id')
train_t.head(10)

,tweet_exp,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you are drunk!!! @USER #MA...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
97670,@USER Liberals are all Kookoo !!!,OFF,TIN,OTH
77444,@USER @USER Oh noes! Tough shit.,OFF,UNT,NaN
52415,@USER was literally just talking about this lo...,OFF,TIN,GRP
45157,@USER Buy more icecream!!!,NOT,NaN,NaN
